In [20]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
import scipy.interpolate as scin
from copy import *
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
from ret import (
    util,
    egm_step
)
from aux import create_container

In [22]:
import pickle
with open('m0_value.pkl', 'rb') as file : 
    m0_value = pickle.load(file)

with open('m0_policy.pkl', 'rb') as file : 
    m0_policy = pickle.load(file)


# Model parameters (default)

In [23]:
# Number of periods (fist period is t=1) 
Tbar = 25

# Number of grid points over assets
ngridm = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 50

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
duw = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16

In [24]:
# Initialize grids
quadstnorm = scps.norm.ppf(ps_roots(n_quad_points)[0])
quadw = ps_roots(n_quad_points)[1]

In [25]:
savingsgrid = np.linspace(0, mmax, ngridm)

In [43]:
policy_test, value_test = create_container(ngridm, Tbar, mmax, theta, duw)

In [36]:
for period in [23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]:
    value, policy = egm_step(value, policy, savingsgrid, quadstnorm, period, Tbar, ngridm, cfloor, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, quadw)

In [13]:
for point in [23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9,  8, 7, 6, 5, 4, 3, 2, 1, 0]:
    np.testing.assert_almost_equal(m0_value[:, 0, 1, point].T[~np.isnan(m0_value[:, 0, 1, point])], value[point][1].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 1, 1, point].T[~np.isnan(m0_value[:, 1, 1, point])], value[point][1].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 0, 1, point].T[~np.isnan(m0_policy[:, 0, 1, point])], policy[point][1].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 1, 1, point].T[~np.isnan(m0_policy[:, 1, 1, point])], policy[point][1].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 0, 0, point].T[~np.isnan(m0_value[:, 0, 0, point])], value[point][0].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_value[:, 1, 0, point].T[~np.isnan(m0_value[:, 1, 0, point])], value[point][0].T[1], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 0, 0, point].T[~np.isnan(m0_policy[:, 0, 0, point])], policy[point][0].T[0], decimal=8)
    np.testing.assert_almost_equal(m0_policy[:, 1, 0, point].T[~np.isnan(m0_policy[:, 1, 0, point])], policy[point][0].T[1], decimal=8)
